### Import libraries

In [1]:
# from libs.Dataset import *
# from libs.SiameseNetwork import *
import matplotlib.pyplot as plt
import dill # allow to save notebook session, useful for test https://towardsdatascience.com/how-to-restore-your-jupyter-notebook-session-dfeadbd86d65
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import Callback, progress
from torch.optim import SGD
from torch import nn
import torch
from tqdm import tqdm
import faiss


In [2]:
# For local execution
data_dir = "dataset/"
path_gdrive = ""
device = "cuda" if torch.cuda.is_available() else "cpu"
img_size = 96

### Loading Triplet Trashbin Pytorch Lighting Data Module

In [3]:
# Init our data pipeline
dm = TripletTrashbinDataModule(data_dir=data_dir,  path_gdrive=path_gdrive, img_size=img_size)
# To access the x_dataloader we need to call prepare_data and setup.
dm.prepare_data()
dm.setup()

Do nothing on prepare_data
---Enter in setup---
***fit***
***test***


In [4]:
val_samples = next(iter(dm.val_dataloader()))
val_imgs, val_labels = val_samples[0], val_samples[1]
print("Required shape: val_imgs: {} val_shapes: {}".format(val_imgs.shape, val_labels.shape))

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/danilo/GitHub/deep-learning/venv/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/Users/danilo/GitHub/deep-learning/venv/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    return self.collate_fn(data)
  File "/Users/danilo/GitHub/deep-learning/venv/lib/python3.9/site-packages/torch/utils/data/_utils/collate.py", line 172, in default_collate
    return [default_collate(samples) for samples in transposed]  # Backwards compatibility.
  File "/Users/danilo/GitHub/deep-learning/venv/lib/python3.9/site-packages/torch/utils/data/_utils/collate.py", line 172, in <listcomp>
    return [default_collate(samples) for samples in transposed]  # Backwards compatibility.
  File "/Users/danilo/GitHub/deep-learning/venv/lib/python3.9/site-packages/torch/utils/data/_u

Print Train Dataset with matplotlib

In [ ]:
plt.figure(figsize=(16,4))
for i, idx in enumerate(np.random.choice(len(dm.trb_train), 6)):
    ax = plt.subplot(3,10,i+1)
    ax.set_title('Tripla {}'.format(idx))
    
    plt.imshow(dm.trb_train[i][0].T)
    plt.subplot(3,10,i+11)
    # plt.title('Class: {}'.format(dm.trb_train[i][3]))
    plt.imshow(dm.trb_train[i][1].T)
    # plt.title('Class: {}'.format(dm.trb_train[i][4]))
    plt.subplot(3,10,i+21)
    plt.imshow(dm.trb_train[i][2].T)
    # plt.title('Class: {}'.format(dm.trb_train[i][5]))

plt.subplots_adjust(hspace=0.1)
plt.show()

### Test pretrained squeezeNet v2

In [ ]:
from torchvision.models import squeezenet1_1

squezenet_fe = squeezenet1_1(pretrained=True)

# rimuovo classificatore finale, imposto il modulo identità al posto del classificatore
squezenet_fe.classifier = nn.Identity()

print("Dimensione del vettore di feature estratto per immagine {} -> {}".format(torch.zeros(1,3,64,64).shape, squezenet_fe(torch.zeros(1,3,64,64)).shape))

In [ ]:
triplet_squeezenet1_1 = TripletNetworkTask(squezenet_fe)
logger = TensorBoardLogger("metric_logs", name="siamese_squeezenet1_1_64")

In [ ]:
# TODO: Aggiungi test delle performance non ancora ottimizzate con questo modello!

In [ ]:
# TSNE

def extract_representations(model, loader, device):
    model.eval()
    model.to(device)
    representations, labels = [], []
    for batch in loader:
        x = batch[0].to(device)
        rep = model(x)
        rep = rep.detach().to(device).numpy()
        labels.append(batch[1])
        representations.append(rep)
    return np.concatenate(representations), np.concatenate(labels)

In [ ]:
dm.test_dataloader()

In [ ]:
test_rep, test_labels = extract_representations(triplet_squeezenet1_1.embedding_net, dm.test_dataloader(), device)

In [ ]:
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt

selected_rep = np.random.choice(len(test_rep), 1000 )
selected_test_rep = test_rep[selected_rep]
selected_test_lab = test_labels[selected_rep]

tsne = TSNE(2)
rep_tsne = tsne.fit_transform(selected_test_rep)

plt.figure(figsize=(8,6))
for c in np.unique(selected_test_lab):
    print(c)
    print(rep_tsne[0,0])
    print(rep_tsne[c, 0])
    print(rep_tsne[selected_test_lab == c, 0])
    print(rep_tsne[selected_test_lab == c, 1])
    plt.plot(rep_tsne[selected_test_lab == c, 0], rep_tsne[selected_test_lab==c, 1], 'o', label=c)
plt.legend()
plt.show()

In [ ]:
trainer = pl.Trainer(gpus=0,
                    max_epochs=1,
                    callbacks=[progress.TQDMProgressBar()],
                    logger=logger,
                    accelerator="auto",
                    )

trainer.fit(model=triplet_squeezenet1_1, datamodule=dm)

In [ ]:
loaded_trainer = pl.Trainer(
    gpus=0,
    logger=logger,
    max_epochs=2,
    progress_bar_refresh_rate=0,
    accelerator="auto",
    resume_from_checkpoint=""
    )


# TODO:
loaded_trainer.fit(model=triplet_squeezenet1_1, datamodule=dm, ckpt_path='metric_logs/siamese_squeezenet1_1_64/version_1/checkpoints/epoch=0-step=206.ckpt')